In [2]:
## ideas which model to use:
## https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis
## https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment?

In [3]:
## dataset for training
##https://huggingface.co/datasets/mteb/tweet_sentiment_extraction

# H1

## H2

### H3

#### H4

## Import statements

In [6]:
import csv
import pandas as pd

## Import emoji dataset from hugging face

In [7]:
from datasets import load_dataset

dataset = load_dataset("arbml/emoji_sentiment_lexicon")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Convert dataset to pd dataframe

In [3]:
# Zugriff auf den Trainingsdatensatz
train_dataset = dataset['train']

# Konvertieren Sie den Trainingsdatensatz in ein Pandas DataFrame
df = pd.DataFrame(train_dataset)

df.to_csv('../data/emoji_dataset.csv', index=False)

In [8]:
df.head(15)

NameError: name 'df' is not defined

## Test Converting emojis to raw text

In [17]:
# test = ["GARBAGE COMPANY 🗑🗑🗑", "I love this 😍❤️🙌", "This shit is another level ❤️"]
# # Erstellen einer Funktion, um Emojis in einem Text durch ihre Unicode-Namen zu ersetzen
# def ersetze_emojis(text):
#     for index, row in df.iterrows():
#         emoji = row['Emoji']
#         unicode_name = row['Unicode_Name']
#         text = text.replace(emoji, unicode_name)
#     return text

# bereinigte_kommentare = [ersetze_emojis(kommentar) for kommentar in test]

# print(bereinigte_kommentare)

['GARBAGE COMPANY 🗑🗑🗑', 'I love this  Smiling Face With Heart-Eyes  Red Heart ️ Raising Hands ', 'This shit is another level  Red Heart ️']


## Clean text of emojis and insert raw text


### Load emoji data from data/... .csv

In [11]:
# Laden Sie den Datensatz mit den Unicode-Namen der Emojis
emoji_data = {}
with open('../data/emoji_dataset.csv', mode='r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        emoji_data[row[2]] = row[3]

### Process texts and substitute emojis

In [10]:
def ersetze_emojis(text):
    for emoji, name in emoji_data.items():
        text = text.replace(emoji, name)
    return text

In [13]:
def clean_text(filepath):
    df_original = pd.read_csv(filepath)
    df_bereinigt = df_original.copy()  
    df_bereinigt['Text'] = df_bereinigt['Text'].apply(ersetze_emojis)

    return df_bereinigt

In [15]:
### bereinige text mit emojis
dataframe_bereinigt = clean_text("../files/oatly.csv")

In [16]:
dataframe_bereinigt

,ID,Text,Is_Activism,Category,Post_Id,Like_Count
0,1,I'm tired of seeing brands and influencers was...,False,NaN,CvfAOApNdUM,1
1,2,It’s driving me crazy that the triangle ‘A’ lo...,False,NaN,CvfAOApNdUM,8
2,3,omfg,False,NaN,CvfAOApNdUM,2
3,4,This sound scraches my brain. I like it Glass...,False,NaN,CvfAOApNdUM,0
4,5,This is so cool and oddly soothing! We love oa...,False,NaN,CvfAOApNdUM,0
...,...,...,...,...,...,...
703,704,As much as I like oatmilk more than cow milk. ...,False,Environmental,CsGiGN0R_Y9,0
704,705,Red Heart ️ It,False,Environmental,CsGiGN0R_Y9,0
705,706,or actually cost to produce/cost to buy!! Fac...,False,Environmental,CsGiGN0R_Y9,0
706,707,Raising Hands,False,Environmental,CsGiGN0R_Y9,0


## Analysing with first model (cardiffnlp/twitter-xlm-roberta-base-sentiment)


In [17]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.5927, 0.5257, 0.8787],
        [0.9849, 0.0781, 0.0509],
        [0.9503, 0.9423, 0.3538],
        [0.9962, 0.0812, 0.4980],
        [0.8752, 0.0305, 0.4194]])


## Train the model

In [26]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

text = "Ich hasse dieses produkt, bitte nie mehr werbung dafür machen"
text = preprocess(text)
print(text)
encoded_input = tokenizer(text, return_tensors='tf')

output = model(encoded_input)
scores = output[0][0].numpy()
scores = softmax(scores)
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")



All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ich hasse dieses produkt, bitte nie mehr werbung dafür machen
1) neutral 0.387
2) negative 0.3623
3) positive 0.2507


## Test the model

In [31]:
from transformers import pipeline
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
result = sentiment_task("I hate this company!")
print(result)



[{'label': 'negative', 'score': 0.9490166902542114}]


In [32]:
result = sentiment_task("I hate this company!")
print(result)

[{'label': 'negative', 'score': 0.9490166902542114}]


## Test the model on different data

### Load model and save it locally

In [39]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"
# Speichere den Tokenizer an einem anderen Ort
tokenizer.save_pretrained(MODEL)

tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)






1) negative 0.9064
2) neutral 0.0601
3) positive 0.0335


### Test model on whatever string you want

In [42]:
text = "Good night 😊"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) positive 0.7673
2) neutral 0.2015
3) negative 0.0313


## Get The Test Comments

In [1]:
import requests
import pandas as pd

# URL zur JSON-Datenquelle
url = "https://datasets-server.huggingface.co/rows?dataset=tweet_eval&config=sentiment&split=validation&offset=0&limit=100"

# Die Anfrage an die URL senden
response = requests.get(url)

# Überprüfen, ob die Anfrage erfolgreich war (Statuscode 200)
if response.status_code == 200:
    # JSON-Daten aus der Antwort extrahieren
    json_data = response.json()
    
    # Erstelle ein leeres Pandas DataFrame mit den gewünschten Spalten 'text' und 'label'
    df = pd.DataFrame(columns=['text', 'label'])
    
    # Schleife durch die Zeilen in den JSON-Daten und füge sie dem DataFrame hinzu
    for row in json_data['rows']:
        text = row['row']['text']
        label = row['row']['label']
        df = pd.concat([df, pd.DataFrame({'text': [text], 'label': [label]})], ignore_index=True)
    
    # Jetzt enthält der DataFrame alle Zeilen aus den JSON-Daten
    # Du kannst diesen DataFrame verwenden oder ihn in eine Datei speichern
    # Zum Beispiel, um ihn als CSV-Datei zu speichern:
    #df.to_csv("dataset.csv", index=False)
else:
    print("Fehler beim Abrufen der Daten:", response.status_code)


In [3]:
df.head(150)

,text,label
0,Dark Souls 3 April Launch Date Confirmed With ...,1
1,"""National hot dog day, national tequila day, t...",2
2,When girls become bandwagon fans of the Packer...,0
3,@user I may or may not have searched it up on ...,1
4,Here's your starting TUESDAY MORNING Line up a...,1
...,...,...
95,I hope u all have a good day bc it's Friday an...,2
96,"""I'm still not over Kendrick's """"Black Friday""...",2
97,"@user Welcome to the 21st century, cuz! Ging...",2
98,"@user Dear Taimouraga, Thank you for contactin...",2


In [4]:
import requests
import pandas as pd

# Anfangs-Offset und Schrittgröße für die Abfrage
start_offset = 0
step = 100  # Die Anzahl der Einträge, die pro Schritt abgerufen werden sollen

# URL zur JSON-Datenquelle
url_template = "https://datasets-server.huggingface.co/rows?dataset=tweet_eval&config=sentiment&split=validation&offset={offset}&limit={limit}"

# Erstelle ein leeres Pandas DataFrame mit den gewünschten Spalten 'text' und 'label'
df = pd.DataFrame(columns=['text', 'label'])

while True:
    # Konstruiere die URL mit dem aktuellen Offset
    url = url_template.format(offset=start_offset, limit=step)

    # Die Anfrage an die URL senden
    response = requests.get(url)

    # Überprüfen, ob die Anfrage erfolgreich war (Statuscode 200)
    if response.status_code == 200:
        # JSON-Daten aus der Antwort extrahieren
        json_data = response.json()

        # Schleife durch die Zeilen in den JSON-Daten und füge sie dem DataFrame hinzu
        for row in json_data['rows']:
            text = row['row']['text']
            label = row['row']['label']
            df = pd.concat([df, pd.DataFrame({'text': [text], 'label': [label]})], ignore_index=True)

        # Wenn weniger Einträge abgerufen wurden als angefordert, brechen Sie die Schleife ab
        if len(json_data['rows']) < step:
            break

        # Erhöhe den Offset für den nächsten Schritt
        start_offset += step
    else:
        print("Fehler beim Abrufen der Daten:", response.status_code)
        break

# Jetzt enthält der DataFrame alle Zeilen aus den JSON-Daten
# Sie können diesen DataFrame verwenden oder ihn in eine Datei speichern
# Zum Beispiel, um ihn als CSV-Datei zu speichern:
# df.to_csv("dataset.csv", index=False)


In [7]:
df.head(2005)

,text,label
0,Dark Souls 3 April Launch Date Confirmed With ...,1
1,"""National hot dog day, national tequila day, t...",2
2,When girls become bandwagon fans of the Packer...,0
3,@user I may or may not have searched it up on ...,1
4,Here's your starting TUESDAY MORNING Line up a...,1
...,...,...
1995,"""LONDON (AP) """" Prince George celebrates his s...",1
1996,Harper's Worst Offense against Refugees may be...,1
1997,Hold on... Sam Smith may do the theme to Spect...,2
1998,Gonna watch Final Destination 5 tonight. I alw...,1


In [ ]:
df.to_csv("../data/val_data.csv", index=False)